Animal Clustering

In [18]:
# Load libraries
import pandas as pd
import numpy as np
import re
import sklearn

In [10]:
# Load in data
df = pd.read_csv("20AnimalsDataset.csv")
df.head()

,ID,animal,paragraph
0,OBS001,dogs,Dogs come in a wide range of sizes and breeds....
1,OBS002,dogs,They are known for their loyalty and companion...
2,OBS003,dogs,They are known for their loyalty and companion...
3,OBS004,dogs,Many families around the world keep them as pe...
4,OBS005,dogs,Many families around the world keep them as pe...


In [12]:
# Extract question

# Function to extract the question (first sentence ending with a '?')
def extract_question(text):
    # This finds all sentences ending in '?'
    questions = re.findall(r'[^.?!]*\?', text)
    if questions:
        return questions[0].strip() # .strip() removes leading space in question
    else:
        return None

# Apply to DataFrame
df['question'] = df['paragraph'].apply(extract_question)

# View result
print(df[['paragraph', 'question']])

                                            paragraph  \
0   Dogs come in a wide range of sizes and breeds....   
1   They are known for their loyalty and companion...   
2   They are known for their loyalty and companion...   
3   Many families around the world keep them as pe...   
4   Many families around the world keep them as pe...   
5   Many people adore them for their mysterious be...   
6   Many people adore them for their mysterious be...   
7   Many people adore them for their mysterious be...   
8   Their purring can be quite soothing to humans....   
9   They often nap for long hours during the day. ...   
10  They communicate through vocalizations and ges...   
11  Monkeys live in forests, mountains, and savann...   
12  Monkeys live in forests, mountains, and savann...   
13  Monkeys are playful and curious by nature. Are...   
14  They live in social groups with hierarchies. M...   
15  They are known for their strength and grace. M...   
16  They often form strong bond

In [19]:
# Cluster ambiguous questions
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

# 1. Convert questions to a list
questions = df['question'].tolist()

# 2. Convert to embeddings using SBERT
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(questions, show_progress_bar=True)

# 3. Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
df['ambig_cluster'] = kmeans.fit_predict(embeddings)

C:\Users\irisa\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\irisa\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular

In [21]:
df

,ID,animal,paragraph,question,ambig_cluster
0,OBS001,dogs,Dogs come in a wide range of sizes and breeds....,Are they more vocal when they're hungry or exc...,1
1,OBS002,dogs,They are known for their loyalty and companion...,How sensitive are they to environmental changes?,0
2,OBS003,dogs,They are known for their loyalty and companion...,What kind of enrichment helps them feel more s...,2
3,OBS004,dogs,Many families around the world keep them as pe...,How do they show signs of trust toward their c...,2
4,OBS005,dogs,Many families around the world keep them as pe...,What do they typically do when they feel threa...,2
5,OBS006,cats,Many people adore them for their mysterious be...,Are sudden noises always startling to them?,1
6,OBS007,cats,Many people adore them for their mysterious be...,Why do they sometimes follow people around?,3
7,OBS008,cats,Many people adore them for their mysterious be...,Why might they pace in new environments?,0
8,OBS009,cats,Their purring can be quite soothing to humans....,Is pacing a sign of anticipation or anxiety?,0
9,OBS010,cats,They often nap for long hours during the day. ...,Do they show preference for certain people ove...,3
